<div align ="right">Thomas Jefferson University <b>COMP 102</b>: Intro to Scientific Computing</div>

# Leslie matrices: Numpy in biology

## A quick review of discrete population growth

In gen bio you learned about discrete population growth. Discrete population growth occurs when an organism breeds at regular intervals (typically once per year). 

With discrete growth, population size at a point in the future depends on the current population size multiplied by the rate of population growth, $\lambda$. Let N<sub>0</sub> be the initial population size, N<sub>1</sub> be the population size after a year, ... or generally N<sub>t</sub> will be the population size after t years and N<sub>t+1</sub> will be the population size the year after that.

$\lambda$ is calculated as the difference between the per-capita birth rate and the per-capita death rate. 

&emsp;&emsp;&emsp;&emsp;A population of 100 individuals produces 20 offspring and suffers 10 deaths

&emsp;&emsp;&emsp;&emsp; In the next generation there will be 110 individuals

&emsp;&emsp;&emsp;&emsp; $ R = \frac{N+B-D}{N} $

&emsp;&emsp;&emsp;&emsp; In this example $\lambda = 1.1 $ 

For every time t, the population at the next time step t+1 will simply be the product of R and the current population:

&emsp;&emsp;&emsp;&emsp; $ N_{t+1} =  \lambda N_t  $

Over any given time interval from $t=0$ to $t=x$, this generalizes to:

&emsp;&emsp;&emsp;&emsp; $ N_{x} = \lambda^{x}N_0   $

Here are a couple of versions of what this might look like in python:

In [ ]:
# Here is the stepwise equation for population growth

N_0 = 100     # N0 = population size at time zero
R = 1.1       # R = lambda

N_1 = 100 * R
N_2 = N_1 * R
N_3 = N_2 * R

print(N3)

In [ ]:
# Create a function discrete that calculates population size for any
## number of generations, lambda (R), and starting population size

def discrete(x, R, N0):
    Nx = N0 * R**x      
    return(Nx)  
                       
print(discrete(3, 1.1, 100))

We can integrate the output of this function into another function for drawing a plot, so that we can visualize the consequences of using different growth parameters. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def discrete_plot(x, R, N0):
    N = []
    t = np.linspace(0,x,x+1)
    for time in t:
        N.append(discrete(time, R, N0))

    plt.plot(t,N, color = 'gray')
    plt.xlabel('generations', size=14)
    plt.ylabel('population size', size=14)
    plt.plot(t, N, 'o',color='slateblue')
    plt.show()

discrete_plot(10,1.1,100)

## Growth in age-structured populations

In the population model above, we assumed that the only determinate of population growth was how many individuals existed in the population, and that they all contributed equally to population growth. In an age-structured population model, we consider that individuals may exist within different age classes within the population, and that individuals in these age classes will have different reproductive contributions and survival rates. For example, a juvenile individual may not contribute reproductive success to the population, and might be likely to die at a higher rate than an adult. 

In an age-structured population, the population will consist of individuals in distinct age classes. For very well known species (like humans) we might use actual yearly age. But for our model let's assume a population of an animal that exists in discreet juvenile, subadult, and adult stages so that we have three separate age classes. So at any time *t*:

&emsp;&emsp;&emsp;&emsp; $ N_{t} =  N_{0,t} + N_{1,t} + N_{2,t} + N_{3,t} $

Each of these age classes will have a fecundity, called $F_0$, $F_1$, etc., which is the product of the expected offspring produced by any individual in this age class over the time interval. 

Each age class will also have a probability of survival, $S_0$, $S_1$, etc.,which is the probability that the organism survives to move on to the next age class.

In order to predict population change over time we want to do the following - start with a vector that represents the population age structure at time $t$, and predict the population size and age structure at time $t+1$:

&emsp;&emsp;&emsp;&emsp; $\left[\begin{array}{cccc}
N_{0,t} \\
N_{1,t} \\
N_{2,t} \\
N_{3,t} \\
\end{array}\right]$
$\Longrightarrow$
$\left[\begin{array}{cccc}
N_{0,t+1} \\
N_{1,t+1} \\
N_{2,t+1} \\
N_{3,t+1} \\
\end{array}\right]$

Let's think about what it would take to do this. To generate $N_{0,t}$ we need to figure out how many offspring will be produced by the individuals in each age class. This of course will be the product of fecundity and the number of individuals in each age class. 

&emsp;&emsp;&emsp;&emsp;$N_{0,t+1} = F_0N_{0,t} + F_1N_{1,t} + F_2N_{2,t} + F_3N_{3,t}$

Likewise, the number of individuals in each of the upper age classes will be the product of survival and the number of individuals in the previous age class:

&emsp;&emsp;&emsp;&emsp;$N_{1,t+1} = S_0N_{0,t}$

&emsp;&emsp;&emsp;&emsp;$N_{2,t+1} = S_1N_{1,t}$

&emsp;&emsp;&emsp;&emsp;$N_{3,t+1} = S_2N_{2,t}$

We can use linear algebra to achieve this result! We will organize our F and S values into a matrix that has the following values. This is known as the Leslie matrix:

&emsp;&emsp;&emsp;&emsp;$\left[\begin{array}{cccc}
F_{0} & F_{1}  & F_{2}  & F_{3} \\
S_0 & 0 & 0 & 0 \\
0 & S_1 & 0 & 0 \\
0 & 0 & S_2 & 0 \\
\end{array}\right]$

Why is this arrangement important? The rules of linear algebra work like this. We can multiply a matrix by a vector using the following rules

&emsp;&emsp;&emsp;&emsp;$\left[\begin{array}{cccc}
a & b  & c  & d \\
e & f & g & h \\
i & j & k & l \\
m & n & o & p \\
\end{array}\right]$
X
$\left[\begin{array}{cccc}
W \\
X \\
Y \\
Z \\
\end{array}\right]$
=
$\left[\begin{array}{cccc}
aW + bX + cY + dZ \\
eW + fX + gY + hZ \\
iW + jX + kY + lZ \\
mW + nX + oY + pZ \\
\end{array}\right]$

So if we multiply the Leslie matrix time the population vector we will get:

&emsp;&emsp;&emsp;&emsp;$\left[\begin{array}{cccc}
F_{0} & F_{1}  & F_{2}  & F_{3} \\
S_0 & 0 & 0 & 0 \\
0 & S_1 & 0 & 0 \\
0 & 0 & S_2 & 0 \\
\end{array}\right]$
X
$\left[\begin{array}{cccc}
N_{0,t} \\
N_{1,t} \\
N_{2,t} \\
N_{3,t} \\
\end{array}\right]$
=
$\left[\begin{array}{cccc}
F_0N_{0,t} + F_1N_{1,t} + F_2N_{2,t} + F_3N_{3,t} \\
S_0N_{0,t} + 0 + 0 + 0\\
0 + S_1N_{1,t} + 0 + 0\\
0 + 0 + S_2N_{1,t} + 0\\
\end{array}\right]$

Which, using the equations above, simplifies to:


&emsp;&emsp;&emsp;&emsp;$\left[\begin{array}{cccc}
F_{0} & F_{1}  & F_{2}  & F_{3} \\
S_0 & 0 & 0 & 0 \\
0 & S_1 & 0 & 0 \\
0 & 0 & S_2 & 0 \\
\end{array}\right]$
X
$\left[\begin{array}{cccc}
N_{0,t} \\
N_{1,t} \\
N_{2,t} \\
N_{3,t} \\
\end{array}\right]$
=
$\left[\begin{array}{cccc}
N_{0,t+1} \\
N_{1,t+1} \\
N_{2,t+1} \\
N_{3,t+1} \\
\end{array}\right]$

If we repeat this calculation multiple times, we can calculate the population at any future time x, but there is no simple exponential form of this equation, as it relies on the special multiplication rules of linear algebra. Therefore, this is a problem that we might want to solve computationally. 

&emsp;&emsp;&emsp;&emsp;$\left[\begin{array}{cccc}
F_{0} & F_{1}  & F_{2}  & F_{3} \\
S_0 & 0 & 0 & 0 \\
0 & S_1 & 0 & 0 \\
0 & 0 & S_2 & 0 \\
\end{array}\right]$
X
$\left[\begin{array}{cccc}
N_{0,x-1} \\
N_{1,x-1} \\
N_{2,x-1} \\
N_{3,x-1} \\
\end{array}\right]$
=
$\left[\begin{array}{cccc}
N_{0,x} \\
N_{1,x} \\
N_{2,x} \\
N_{3,x} \\
\end{array}\right]$


## Setting the leslie matrix in python

So the parameters we need to set to do this calculation are:

* An array of values for the population size at time 0. 
* The F values for each age class
* The S values for each transition between age classes 

Let's set up a situation where we have low survival and reproduction in the youngest age classes, and relatively high reproductive rates and survival in the middle age classes before a steep decline in reproduction in the oldest age class.

Rather than manually type out an array, let's write a function to create a leslie matrix from a set of F and S values.

In [ ]:
Nt0_arr = np.array([100,100,100,100])      # array for population at time zero

F = np.array([0,2.5,3.5,1.1])           
S = np.array([0.15,0.95,0.9])

# define a function to make a leslie matrix from F and S

def leslie(F,S):
    L = np.array([F, 
                  [S[0],0,0,0],
                  [0,S[1],0,0],
                  [0,0,S[2],0]])
    return(L)

L = leslie(F,S)
print(L)
print()
print(type(L))

Now that we have our leslie matrix, we can multiply to figure out the population array for the next generation, then the next, then the next. Notice that we can't use a standard `*` multiplication. We can use `matmul()` to indicate that what we want here is specifically matrix multiplication following the rules of linear algebra. 

In [ ]:
Nt1_arr = np.matmul(L, Nt0_arr)
Nt2_arr = np.matmul(L, Nt1_arr)
Nt3_arr = np.matmul(L, Nt2_arr)
Nt4_arr = np.matmul(L, Nt3_arr)

print('time 0: ',Nt0_arr.sum(),Nt0_arr)
print('time 1: ',Nt1_arr.sum(),Nt1_arr)
print('time 2: ',Nt2_arr.sum(),Nt2_arr)
print('time 3: ',Nt3_arr.sum(),Nt3_arr)
print('time 4: ',Nt4_arr.sum(),Nt4_arr)


Note that the age structure in this model gives it some unusual dynamics compared to the simple population we graphed above. Is it increasing or decreasing in size? 

## Exercise

Define a function that will calculate population size and age distribution for an age structured population at a future date *x*. The function will need to take:
* An array representing the distribution of the population at time 0
* An array of age-specific fecundities
* An array of age-specific survival
* The number of generations to be considered


In [ ]:
#
## Your code here
#

## Bonus Exercise 
Can you incorporate plotting into your functino (using the template provided in the example above for the making of the plot itself) in order to track the change in population over time?

In [ ]:
#
## Your code here
#

![Alt text that will appear on mouseover](images/TJU_logo_dummy_image.png "Dummy image")